In [ ]:
# This is for Japan data source, Chubu Electric Power.
# Author: Sang-Won Yu

# NOTE: The data source starts from April 2016 until now. There are separate CSVs for each year. 
# Each CSV starts from April, and ends in March the following year, because fiscal year in Japan
# is from April through March. Chubu Electric Power serves the Chubu region, which is located in
# the middle of Honshu Island, the main island of Japan. 

import pandas as pd
import numpy as np
from datetime import datetime

def read_chubu_csv():
  combined_data = pd.DataFrame()
  for year in range(2016, datetime.now().year + 1):
    csv = pd.read_csv('https://powergrid.chuden.co.jp/denki_yoho_content_data/' +
                      str(year) + '_areabalance_current_term.csv', header = 4,
                      encoding = 'shift_jis')
    combined_data = combined_data.append(csv, ignore_index=True)
  # Translate Japanese column names to English
  combined_data.columns = ['Date', 'Time', 'Area_Demand', 'Nuclear', 'Thermal',
                          'Hydraulic', 'Geothermal', 'Biomass', 'Solar(Actual)',
                          'Solar(Output_Control)', 'Wind(Actual)', 'Wind(Output_Control)',
                          'Pumped_Hydro', 'Interconnector']

  # assign units and region
  combined_data['Region'] = 'Chubu'
  combined_data['Unit'] = 'MWh'

  # combine columns 'DATE' and 'TIME' to make a datetime object
  combined_data['Date']=pd.to_datetime(combined_data['Date'] + ' ' + combined_data['Time'], format='%Y/%m/%d %H:%M')
  combined_data.drop('Time', axis=1, inplace=True)
  combined_data.rename(columns={"Date": "Date_Time"}, inplace=True)

  # get demand data into one df
  demand_df = combined_data[['Date_Time', 'Region', 'Unit', 'Area_Demand']].copy()
  demand_df.sort_values(by=['Date_Time'],ascending=False, inplace=True)

  # get supply data into another df
  supply_df = combined_data
  supply_df.drop('Area_Demand', axis=1, inplace=True)
  # Pivot "wide" to "long" format
  supply_df = pd.melt(combined_data, id_vars=['Date_Time','Region', 'Unit'], var_name='Fuel_Type', value_name='Supply')
  supply_df.sort_values(by=['Date_Time','Fuel_Type'], ascending=False, inplace=True)
  
  return demand_df, supply_df

chubu_data = read_chubu_csv()

In [ ]:
chubu_data[0]

,Date_Time,Region,Unit,Area_Demand
53303,2022-04-30 23:00:00,Chubu,MWh,9959
53302,2022-04-30 22:00:00,Chubu,MWh,10247
53301,2022-04-30 21:00:00,Chubu,MWh,10739
53300,2022-04-30 20:00:00,Chubu,MWh,11335
53299,2022-04-30 19:00:00,Chubu,MWh,11658
...,...,...,...,...
4,2016-04-01 04:00:00,Chubu,MWh,12227
3,2016-04-01 03:00:00,Chubu,MWh,12362
2,2016-04-01 02:00:00,Chubu,MWh,12273
1,2016-04-01 01:00:00,Chubu,MWh,12129


In [ ]:
chubu_data[1]

,Date_Time,Region,Unit,Fuel_Type,Supply
479735,2022-04-30 23:00:00,Chubu,MWh,Wind(Output_Control),0
426431,2022-04-30 23:00:00,Chubu,MWh,Wind(Actual),145
106607,2022-04-30 23:00:00,Chubu,MWh,Thermal,6842
373127,2022-04-30 23:00:00,Chubu,MWh,Solar(Output_Control),0
319823,2022-04-30 23:00:00,Chubu,MWh,Solar(Actual),0
...,...,...,...,...,...
0,2016-04-01 00:00:00,Chubu,MWh,Nuclear,0
533040,2016-04-01 00:00:00,Chubu,MWh,Interconnector,-331
106608,2016-04-01 00:00:00,Chubu,MWh,Hydraulic,704
159912,2016-04-01 00:00:00,Chubu,MWh,Geothermal,0
